In [1]:
import sys
sys.path.append('../')

# was getting weird async error due to using django w/jupyter, so changed this flag...dangerous???
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"]='1'

from collections import defaultdict
import django
django.setup()
from django.apps import apps
from core.models import *
import pandas as pd
import re

def url2doc_id(u):
    r = re.findall('\/d\/(.+?)\/', u)
    if len(r) == 0:
        return None
    else:
        return r[0]

def id2export_url(id_):
    return 'https://docs.google.com/spreadsheets/d/%s/export?format=xlsx&id=%s' % (id_, id_)

def get_sheet_by_name(sheets, name):
    for sheet_name, sheet in sheets.items():
        if sheet_name.lower() == name.lower():            
            return sheet
    return None
    
url = 'https://docs.google.com/spreadsheets/d/1sH0XnBsGZZ9PCzhYE53o4hEClziCNULakF1ZanPFOqA/edit?usp=sharing'
sheets = pd.read_excel(id2export_url(url2doc_id(url)), sheet_name=None)

/var/folders/zd/7chgfkr973b4wjh0pjrgmfs40000gn/T/ipykernel_39824/2167890486.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# read all entities and add them to database.

def create_entity_types(sheets):
    entity_types_df = get_sheet_by_name(sheets, 'Entities')
    entity_types = {}
    for i, e in entity_types_df.iterrows():
        name = e.Type
        e = EntityType(name=name)
        e.save()
        entity_types[name] = e
        print('Added Entity Type', name)
    return entity_types
    
def create_relationship_types(sheets, db):
    relationship_types_df = get_sheet_by_name(sheets, 'Relationships')
    relationship_types = defaultdict(lambda: {})
    for i, r in relationship_types_df.iterrows():
        rt = RelationshipType(source_entity_type=db.entity_types[r['Entity 1']],
                             target_entity_type=db.entity_types[r['Entity 2']],
                             name=r.Name)
        rt.save()
        relationship_types[r['Entity 1']][r.Name] = rt
        print('Added Relationship Type', r['Entity 1'], r.Name)
    return relationship_types

def create_fields(sheets, db):
    # Add all Fields for each entity
    fields = defaultdict(lambda: {})
    for entity_type_name, entity_type in db.entity_types.items():
        sheet = sheets[entity_type_name]
        print('Adding Fields for entity', entity_type_name)
        for c in sheet.columns:
            print('  Field', c)
            if (c in db.relationship_types[entity_type_name] or 
                re.match(r'.*\.[0-9]+$', c)): 
                # skip relationship columns. for duplicate columns, pandas appends .1, .2 etc. 
                # We assume these are relationship columns for now.
                print('       skipping relationship', c)
                continue
            field = Field(entity_type=entity_type,
                          name=c,
                          is_image_field=c==entity_type.image_field_name,
                          is_title_field=c==entity_type.title_field_name)
            field.save()        
            fields[entity_type_name][c] = field
    return fields

def create_entities(sheets, db):
    entities = defaultdict(lambda: {})
    for entity_type_name in db.entity_types:
        print('adding entities', entity_type_name)
        sheet = sheets[entity_type_name]    
        entity_type = db.entity_types[entity_type_name]
        for _, row in sheet.iterrows():
            print('    adding entity', row.Key)
            key = None
            values = []
            for col, val in row.to_dict().items():
                if (col in db.relationship_types[entity_type_name] or 
                    re.match(r'.*\.[0-9]+$', col) or col.lower()=='name'): 
                    continue
                v = Value(field=db.fields[entity_type_name][col], 
                          entity_type=entity_type,
                          value=str(val))
                v.save()
                values.append(v)
                if col.strip().lower() == 'key':
                    key = str(val)
            entity = Entity(entity_type=entity_type, key=key, name=row['Name'])
            entity.save()
            print('adding values', values)
            entity.values.set(values)
            entity.save()
            entities[entity_type_name][key] = entity    
    return entities

def create_relationships(sheets, db):
    relationships = defaultdict(lambda: [])
    for entity_type_name in db.entity_types:
        sheet = sheets[entity_type_name]    
        for _, row in sheet.iterrows():
            for col, val in row.to_dict().items():     
                if pd.isnull(val): # nothing to add
                    continue
                # drop any added .1, .2 etc string added for duplicate rows
                col = re.sub(r'\.[0-9]+$', '', col)
                if col in db.relationship_types[entity_type_name]:
                    relationship_type = db.relationship_types[entity_type_name][col]
                    entity1 = db.entities[entity_type_name][row.Key]
                    entity2_type = relationship_type.target_entity_type.name
                    print(col, relationship_type.name, entity1.key, entity2_type, val)
                    entity2 = db.entities[entity2_type][val]
                    r = Relationship(relationship_type=relationship_type,
                                     source_entity=entity1,
                                     target_entity=entity2).save()    
                    relationships[col] = r
    return relationships
    
class DB:
    # dummy class for in-memory db
    def __init__(self):
        pass

for model in apps.get_models():
    model.objects.all().delete()
db = DB()
db.entity_types = create_entity_types(sheets)
db.relationship_types = create_relationship_types(sheets, db)
# db.relationship_types
db.fields = create_fields(sheets, db)
db.entities = create_entities(sheets, db)
db.relationships = create_relationships(sheets, db)

Added Entity Type Artist
Added Entity Type Artwork
Added Entity Type Patron
Added Entity Type Location
Added Relationship Type Artist MentoredBy
Added Relationship Type Artwork CreatedBy
Added Relationship Type Artwork CommissionedBy
Added Relationship Type Artwork InfluencedBy
Added Relationship Type Artwork LocatedIn
Adding Fields for entity Artist
  Field Key
  Field Name
  Field Born
  Field Died
  Field MentoredBy
       skipping relationship MentoredBy
Adding Fields for entity Artwork
  Field Key
  Field Name
  Field Image URL
  Field CreatedBy
       skipping relationship CreatedBy
  Field CreatedBy.1
       skipping relationship CreatedBy.1
  Field CreatedBy.2
       skipping relationship CreatedBy.2
  Field Start Date
  Field End Date
  Field Medium
  Field InfluencedBy
       skipping relationship InfluencedBy
  Field InfluencedBy.1
       skipping relationship InfluencedBy.1
  Field InfluencedBy.2
       skipping relationship InfluencedBy.2
  Field CommissionedBy
       skip

In [15]:
print('Entity Types')
display([e.name for e in EntityType.objects.all()])

print('\nFields')
display([(f.entity_type.name, f.name) for f in Field.objects.all()])

print('\nRelationship Types')
display([e.name for e in RelationshipType.objects.all()])

print('Entities')
display([(f.entity_type.name, f.key) for f in Entity.objects.all()])

print('Values')
display([(v.entity_type.name, v.field.name, v.value) for v in Value.objects.all()])

print('Relationships')
display([(v.relationship_type.name, v.source_entity.key, v.target_entity.key) for v in Relationship.objects.all()])


Entity Types


['Artist', 'Artwork', 'Patron', 'Location']


Fields


[('Artist', 'Key'),
 ('Artist', 'Name'),
 ('Artist', 'Born'),
 ('Artist', 'Died'),
 ('Artwork', 'Key'),
 ('Artwork', 'Name'),
 ('Artwork', 'Image URL'),
 ('Artwork', 'Start Date'),
 ('Artwork', 'End Date'),
 ('Artwork', 'Medium'),
 ('Patron', 'Key'),
 ('Patron', 'Name'),
 ('Patron', 'Born'),
 ('Patron', 'Died'),
 ('Location', 'Key'),
 ('Location', 'Name'),
 ('Location', 'Lat'),
 ('Location', 'Lon')]


Relationship Types


...

[('Artist', 'Raphael'),
 ('Artist', 'Polidoro'),
 ('Artwork', 'Madonna of Loreto'),
 ('Artwork', 'Polidoro Psyche'),
 ('Patron', 'Julius II'),
 ('Location', 'Rome')]

Values


[('Artist', 'Key', 'Raphael'),
 ('Artist', 'Born', '1483'),
 ('Artist', 'Died', '1520'),
 ('Artist', 'Key', 'Polidoro'),
 ('Artist', 'Born', '1499'),
 ('Artist', 'Died', '1543'),
 ('Artwork', 'Key', 'Madonna of Loreto'),
 ('Artwork',
  'Image URL',
  'https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Rapha%C3%ABl_-_La_Madone_de_Lorette_-_Google_Art_Project.jpg/1024px-Rapha%C3%ABl_-_La_Madone_de_Lorette_-_Google_Art_Project.jpg'),
 ('Artwork', 'Start Date', 'nan'),
 ('Artwork', 'End Date', 'nan'),
 ('Artwork', 'Medium', 'nan'),
 ('Artwork', 'Key', 'Polidoro Psyche'),
 ('Artwork',
  'Image URL',
  'https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/Polidoro_de_Caravaggio_Psyche_and_her_sisters.jpg/1599px-Polidoro_de_Caravaggio_Psyche_and_her_sisters.jpg'),
 ('Artwork', 'Start Date', 'nan'),
 ('Artwork', 'End Date', 'nan'),
 ('Artwork', 'Medium', 'nan'),
 ('Patron', 'Key', 'Julius II'),
 ('Patron', 'Born', '1443'),
 ('Patron', 'Died', '1513'),
 ('Location', 'Key', 'Rome'),


Relationships


[('MentoredBy', 'Polidoro', 'Raphael'),
 ('CreatedBy', 'Madonna of Loreto', 'Raphael'),
 ('CommissionedBy', 'Madonna of Loreto', 'Julius II'),
 ('CreatedBy', 'Polidoro Psyche', 'Polidoro')]

In [3]:
import json
def db2json():
    nodes = []
    for e in Entity.objects.all():
        nodes.append({
            'id': e.pk,
            'key': e.key,
            'name': e.name,
            'entity_type': e.entity_type.name,
            'image_url': e.get_value('Image URL')
        })
    links = []
    for r in Relationship.objects.all():
        links.append({
            'source': r.source_entity.pk,
            'target': r.target_entity.pk,
            'relationship_type': r.relationship_type.name,
            'value': 1
        })
    return json.dumps({'nodes': nodes, 'links': links}).encode('utf-8')
    
import gzip
Graph(compressed_json=gzip.compress(db2json())).save()

In [9]:
Graph.objects.first().get_json()

AttributeError: 'Graph' object has no attribute 'get_json'

In [5]:
e = db.entities['Artwork']['Polidoro Psyche']
e.get_value('Image URL')
# for v in e.values.all():
#     print(v.field.name, v.value)

In [18]:
re.sub('asd'.replace)


Help on built-in function replace:

replace(old, new, count=-1, /) method of builtins.str instance
    Return a copy with all occurrences of substring old replaced by new.
    
      count
        Maximum number of occurrences to replace.
        -1 (the default value) means replace all occurrences.
    
    If the optional argument count is given, only the first count occurrences are
    replaced.

